### Init

In [11]:
from dotenv import load_dotenv
from lib.seed import seed_prefix 
import sys, os 
load_dotenv()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
seed_prefix(seed = 42)

Seed Fix: 42


### [1]-[2]

In [12]:
from lib.dataset import data_split
import pandas as pd 
data_df = pd.read_csv(os.getenv('DATASHEET_PATH'))
data_dir = os.getenv('DATA_DIR')
train_df, test_df = data_split(data_df, split_num = 5)
test_df.to_csv('/home/eiden/eiden/PCOS-roi-classification/v2/data/datasheet_test.csv', encoding = 'utf-8-sig', index = False)
binary_use = False
test_df

,filename,"label|0:양성, 1:중간형, 2:악성",PID
0,0_R009_00001,0,R009
1,0_R014_00001,0,R014
2,0_R017_00001,0,R017
3,0_R019_00001,0,R019
4,0_R028_00001,0,R028
...,...,...,...
587,2_R893_00001,2,R893
588,2_R893_00002,2,R893
589,2_R893_00003,2,R893
590,2_R919_00001,2,R919


In [13]:
from lib.dataset import PCOS_Dataset
import torchvision.transforms as T
from torchvision.transforms import v2

labels = test_df['label|0:양성, 1:중간형, 2:악성']
filenames = test_df['filename']
filepaths = [os.path.join(data_dir, filename) for filename in filenames]
filepaths

transform = T.Compose([
    T.Resize((224, 224)),
    T.Grayscale(num_output_channels = 3),
    T.ToTensor(),
])

In [14]:
from PIL import Image
import torch

def reshape_transform(filepath, label, transform):
    data = Image.open(filepath + '.png')
    if binary_use:
        label = 0 if label == 1 else 1 # 보더라인을 양성에 붙힌 경우 AUC : 0.7246253000224796
        # 다중 뷴류 문제라면 float 타입을 쓰는 경우가 많습니다. (원-핫이 아닌 class index라고 가정)
        label = torch.tensor(label, dtype=torch.float32)
    else:
        # 다중 뷴류 문제라면 long 타입을 쓰는 경우가 많습니다. (원-핫이 아닌 class index라고 가정)
        label = torch.tensor(label, dtype=torch.long)
        
    if transform:
        data = transform(data)
    return data, label

X, label = reshape_transform(filepaths[0], labels[0], transform)

import matplotlib.pyplot as plt
plt.imshow(X.permute(1, 2, 0))


# Model Load

In [ ]:
import torch 
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from models import Model_Loader
model = Model_Loader(model_name = 'convnext' + '_' + 'l', num_classes = 3).to('cuda')
target_layers = [model.features[-1][-1].block] # ConvNext 

/home/eiden/miniconda3/envs/pcos/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Large_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
from models import get_checkpoint_path
ckpt_paths = get_checkpoint_path(
    checkpoint_dir = './log/',
    datetime = "2025-03-04_10-39-40"
)
ckpt_paths


['./log/2025-03-04_10-39-40_fold1.pth',
 './log/2025-03-04_10-39-40_fold2.pth',
 './log/2025-03-04_10-39-40_fold3.pth',
 './log/2025-03-04_10-39-40_fold4.pth',
 './log/2025-03-04_10-39-40_fold5.pth']

#### Optimize, Loss 정의

#### [Settings] Train - Hyper Parmas 정의

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch 
import datetime

In [ ]:

from metric import calculate_metrics, plot_confusion_matrix_from_preds
from lib.pytorch_grad_cam import (
    GradCAM,
    HiResCAM,
    ScoreCAM,
    GradCAMPlusPlus,
    AblationCAM,
    XGradCAM,
    EigenCAM,
    FullGrad
)

from lib.pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from lib.pytorch_grad_cam.utils.image import show_cam_on_image
from visualize import plot_roc_curve


# Testing with K-Fold Grad CAM

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm 
metric_dict = dict()

for fold_num, ckpt_path in enumerate(ckpt_paths):
    model.load_state_dict(torch.load(ckpt_path))
    model.eval()
    val_prob = []
    val_label = []
    
    for filepath, label in tqdm(zip(filepaths, labels), total = len(filepaths), desc = f'Fold_{fold_num+1}'):
        with GradCAM(model = model, target_layers = target_layers) as cam:
            X, label = reshape_transform(filepath, label, transform)
            X, label = X.to('cuda'), label.to('cuda')
            if len(X.shape) == 3: # (C, H, W) -> (B, C, H, W)
                X = X.unsqueeze(0).to('cuda')
            
            cam.batch_size = X.shape[0]
            grayscale_cam = cam(
                input_tensor = X.to('cuda'), 
                # targets=[ClassifierOutputTarget(C의 class index)]
                # targets=None이면 classification score가 가장 높은 클래스에 대한 결과를 보여줌 -> Pred값 보여줌
                targets = [ClassifierOutputTarget(label)],       # 특정 class C에 대한 결과를 확인하려면 아래와 같이 설정
                eigen_smooth = True, # Flipping을 통해 실행시간이 x6으로 늘어나서 물체를 잘 보여주도록 함. 
                aug_smooth = True, #  # Noise를 제거하여 물체를 잘 보여주도록 함.
                )

            y_res_val = cam.outputs.detach() # 모델 출력 1 x Num Classes
            if binary_use:
                y_pred = torch.sigmoid(y_res_val).cpu()
                
            else:
                y_pred = F.softmax(y_res_val, dim = 1) # 확률 값 : 1 x Num Classes
                
            val_prob.append(y_pred.cpu().detach())
            val_label.append(label.cpu().detach().unsqueeze(0))
            
            ##%% Gray Scale CAM
            # 만약 grayscale_cam[i]도 torch.Tensor라면 numpy로 변환 (이미 2D여야 함)
            cam_mask = grayscale_cam.squeeze(0).cpu().numpy() if torch.is_tensor(grayscale_cam.squeeze(0)) else grayscale_cam.squeeze(0) # model output shape
            # img를 numpy로 변환 & scailing img to 0~1
            img = np.asarray(Image.open(filepath + '.png').convert('RGB')) / 255.0
            # cam_mask size scaling to img size
            cam_mask = cv2.resize(cam_mask, (img.shape[1], img.shape[0]))
            
            visualization = show_cam_on_image(img, cam_mask, use_rgb=True, image_weight = 0.6)
            
            ## save to image
            save_dir = ckpt_path.replace('.pth', '')
            os.makedirs(save_dir, exist_ok = True)
            save_path = os.path.join(save_dir, filepath.split('/')[-1] + '.png')
            cv2.imwrite(save_path, cv2.cvtColor(visualization, cv2.COLOR_RGB2BGR))
            # 각 파일 처리 후 사용한 메모리 해제
            del X, grayscale_cam, y_res_val, y_pred
            torch.cuda.empty_cache()
            import gc
            gc.collect()
        
    #%% Validation AUC 계산
    val_pred = torch.cat(val_prob, dim = 0)
    val_label = torch.cat(val_label, dim = 0)
    metric_dict[f'Fold_{fold_num+1}'] = calculate_metrics(labels = val_label, preds = val_pred, binary_use = False)

    class_names = ['양성', '중간형', '악성']
    metric_dict[f'Fold_{fold_num+1}']['Best Threshold'] = plot_roc_curve(true_labels = val_label, pred_probs = val_pred, binary_use = binary_use, class_names = class_names)
    plot_confusion_matrix_from_preds(val_label, val_pred, binary_use=False, class_names=class_names, save_path = ckpt_path.replace('.pth' ,'_CM.png'), normalize=True)
    # 폴드 처리 후도 캐시 비우기
    torch.cuda.empty_cache()
    gc.collect()

metric_dict

Fold_4:  29%|██▉       | 173/592 [04:12<10:12,  1.46s/it]  


KeyboardInterrupt: 